In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import apply_affine_transform
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight


from PIL import Image
import matplotlib.pyplot as plt

from collections import Counter


# Load Dataset

In [ ]:
CSV_FILE_NAME = './data/my_file.csv'
arr = pd.read_csv(CSV_FILE_NAME, header=None)

In [ ]:
# X, y_class, y_reg = arr.iloc[:, :-5].values, arr.iloc[:, -5:-4].values, arr.iloc[:, -4:].values
X, y = arr.iloc[:, :-5].values, arr.iloc[:, -5:].values
X = X.reshape(-1, 200, 200, 1)


# one hot encoding
# y_class = tf.keras.utils.to_categorical(y_class, num_classes=17)

# dataset split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
y_class_test, y_reg_test = y_test[:, -5:-4], y_test[:, -4:]
y_class_train, y_reg_train = y_train[:, -5:-4], y_train[:, -4:]
# print(type(y_class_train))
y_class_train = y_class_train +1
y_class_test = y_class_test +1
# y_class_train = [x+1 for x in y_class_train]
# y_class_test = [x+1 for x in y_class_test]
# print(max(y_class_test))
# print(min(y_class_test))

# if no data augmentation
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_class_train.argmax(axis=1)),
                                                  y=y_class_train.argmax(axis=1))

class_weights_dict = dict(enumerate(class_weights))


In [ ]:
print(y_train.shape, y_test.shape)

# Model

In [ ]:
seqmodel = models.Sequential([
    layers.Input(shape=(200, 200, 1)),
    # layers.Dense(40000),
    # layers.Input(shape=(400, 400, 1)),
    layers.Conv2D(32, 3, activation='relu'),
    # layers.Conv2D(32, 3, kernel_regularizer=l2(0.0001), activation='relu'),
    layers.MaxPooling2D(2),
    layers.Conv2D(64, 3, activation='relu'),
    # layers.Conv2D(64, 3, kernel_regularizer=l2(0.0001), activation='relu'),
    layers.MaxPooling2D(2),
    layers.Flatten(),
    layers.Dropout(0.2),
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.2),
])

#change from sequential to functional to enable branches
input_layer = layers.Input((200,200,1))
prev_layer = input_layer
for layer in seqmodel.layers:
    prev_layer = layer(prev_layer)


outputClass = layers.Dense(17, activation="sigmoid", name="CatOut")(prev_layer)
outputReg = layers.Dense(4, activation="softmax", name ="RegOut")(prev_layer)

#Combine
funcmodel = models.Model(inputs=input_layer, outputs=[outputClass, outputReg])
# funcmodel = models.Model([input_layer], [prev_layer])


reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=5)

# f1_metric = tf.metrics.F1Score(average='macro')
# f1_metric = tf.metrics.F1Score(average='weighted') # if no data augmentation
# mse_metric = tf.metrics.Mean


funcmodel.compile(optimizer=Adam(0.001),
              loss= {'CatOut': 'sparse_categorical_crossentropy', 'RegOut': 'mse'},
              metrics={'CatOut': tf.metrics.SparseCategoricalAccuracy(name='acc'), 'RegOut': 'accuracy'},
              )


funcmodel.summary()

In [ ]:
history = funcmodel.fit(X_train, {'CatOut': y_class_train, 'RegOut': y_reg_train}, epochs=50, validation_data=(X_test, {'CatOut': y_class_test, 'RegOut': y_reg_test}))

In [ ]:
# Convert model history dictionary to DataFrame
df_loss_acc = pd.DataFrame(funcmodel.history.history)

# Select loss columns and rename
df_loss = df_loss_acc[['loss', 'val_loss']].copy()
df_loss.rename(columns={'loss': 'train', 'val_loss': 'validation'}, inplace=True)

# Select accuracy columns and rename
df_acc = df_loss_acc[['RegOut_accuracy', 'val_RegOut_accuracy']].copy()
df_acc.rename(columns={'accuracy': 'train', 'val_accuracy': 'validation'}, inplace=True)

# Select accuracy columns and rename
df_acc2 = df_loss_acc[['CatOut_acc', 'val_CatOut_acc']].copy()
df_acc2.rename(columns={'accuracy': 'train', 'val_accuracy': 'validation'}, inplace=True)

# Plot model loss
df_loss.plot(title='Model loss', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Loss')

# Plot model accuracy
df_acc.plot(title='Model Accuracy (Bounding)', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')

# Plot model accuracy
df_acc2.plot(title='Model Accuracy (Category)', figsize=(12, 8)).set(xlabel='Epoch', ylabel='Accuracy')


In [ ]:
result = funcmodel.evaluate(X_test, y_test_reg)
print("Result:", result)